In [129]:
import os
import datetime

sym_type_dict = {'Composite': 0, 'Module': 1, 'Annotate': 3, 'Pin': 4, 'Border': 5}
a_idx = {'A': 0, 'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 6, 'value': 7}
pin_idx = {'P': 0, 'id': 1, 'x1': 2, 'y1': 3, 'x2': 4, 'y2': 5, 'unk': 6, 'side': 7, 'inverted': 8} # Pin row to string index
lbl_idx = {'L': 0, 'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'unk': 6, 'visible': 7, 'unk2': 8, 'value': 9} # Line row values to string index
vis_dict = {'Hidden': 0, 'Visible': 1}
a_vis_dict = {'Hidden': 0, 'Hidden-wProperty':2, 'Visible': 3, 'Visible-wProperty': 4}
inv_dict = {False: 0, 'False': 0, 'FALSE': 0, 'Inverted': 1, True: 1, 'True': 1, 'TRUE': 1}
side_dict = {'Top': 0, 'Bottom': 1, 'Left': 2, 'Right': 3}
rotation_dict = {'0': 0, '90': 1, '180': 2, '270': 3, 0: 0, 90: 1, 180: 2, 270: 3}
just_dict = {'Upper Left': 1, 'Middle Left': 2, 'Lower Left': 3, 'Upper Center': 4, 'Middle Center': 5, 'Lower Center': 6, 'Upper Right': 7, 'Middle Right': 8, 'Lower Right': 9}
pin_types = ['IN', 'OUT', 'BI', 'TRI', 'OCL', 'OEM', 'POWER', 'GROUND', 'ANALOG']

In [122]:
out_dir = 'Created'
symbol_name = 'test_symbol'

# Defaults
symbol_type = 'Module'

In [130]:
f = open(os.path.join(out_dir, symbol_name) + '.1', 'a')

In [131]:
def create_main_hdr(f):
    hdr = ['V 54']
    hdr += ['K 33671749690 new_symbol']
    hdr += ['F Case']
    hdr += ['|R ' + datetime.datetime.now().strftime('%H:%M:%S_%m-%d-%y')]
    hdr += ['|BORDERTYPESUPPORT']
    hdr += ['Y ' + str(sym_type_dict[symbol_type])]
    hdr += ['D 0 0 2540000 2540000', 'Z 10', 'i 3', '|I 6']

    write_str_list(f, hdr)
        
def mils_to_units(mils):
    return int((mils/100)*254000)

def write_str_list(f, str_list):
    for s in str_list:
        f.write(s)
        f.write('\n')
        
def add_pin(f, pid, x, y, side, inv, num, label, pin_type):
    p_str = get_pin_str(pid, x, y, side, inv)
    
    lbl_x = x
    lbl_y = y
    num_x = x
    num_y = y
    
    just = 'Middle ' + side
    num_just = 'Lower Right'
    
    lbl_delta = 350
    num_delta_x = 200
    if side == 'Left':
        lbl_x += lbl_delta
        num_x += num_delta_x
    elif side == 'Right':
        lbl_x -= lbl_delta
        num_x -= num_delta_x
        num_just = 'Upper Left'
        
    # TODO: Add justification for Top and Bottom pins
    l_str = get_label_str(lbl_x, lbl_y, 100, 0, just, 'Visible', label)
    
    pin_type_str = get_pintype_str(pin_type)
    
    pin_num_str = get_property_str('#', num_x, num_y, 100, 0, num_just, 'Visible', num)
    
    write_str_list(f, [p_str, l_str, pin_type_str, pin_num_str])

def get_pin_str(pid, x, y, side, inv):
    pin_length = mils_to_units(300) # 300 mils or 762000 units
    x1 = mils_to_units(x)
    x2 = mils_to_units(x)
    y1 = mils_to_units(y)
    y2 = mils_to_units(y)
    
    if side == 'Left':
        x2 = x1 + pin_length
    elif side == 'Right':
        x2 = x1 - pin_length
    elif side == 'Top':
        y1 = y2 + pin_length
    elif side == 'Bottom':
        y2 = y1 + pin_length
        
    vals = {pin_idx['P']: 'P', 
            pin_idx['id']: pid,
            pin_idx['x1']: x1,
            pin_idx['y1']: y1,
            pin_idx['x2']: x2,
            pin_idx['y2']: y2,
            pin_idx['unk']: 0,
            pin_idx['side']: side_dict[side],
            pin_idx['inverted']: inv_dict[inv]}
    
    out_str = ''
    for v in vals.values():
        out_str += str(v) + ' '
        
    return out_str.strip()    

def get_label_str(x, y, size, rot, just, vis, val):
    vals = {lbl_idx['L']: 'L', 
            lbl_idx['x']: mils_to_units(x),
            lbl_idx['y']: mils_to_units(y),
            lbl_idx['size']: mils_to_units(size),
            lbl_idx['rotation']: rotation_dict[rot],
            lbl_idx['justification']: just_dict[just],
            lbl_idx['unk']: 0,
            lbl_idx['visible']: vis_dict[vis],
            lbl_idx['unk2']: 0,
           lbl_idx['value']: val}
    
    out_str = ''
    for v in vals.values():
        out_str += str(v) + ' '
        
    return out_str.strip()    

def get_pintype_str(val): 
    assert val in pin_types
    return get_property_str('PINTYPE', 0, 0, mils_to_units(100), 0, 'Middle Left', 'Hidden', val)

def get_property_str(prop, x, y, size, rot, just, vis, val):    
    vals = {a_idx['A']: 'A', 
            a_idx['x']: mils_to_units(x),
            a_idx['y']: mils_to_units(y),
            a_idx['size']: mils_to_units(size),
            a_idx['rotation']: rotation_dict[rot],
            a_idx['justification']: just_dict[just],
            a_idx['visible']: a_vis_dict[vis],
            a_idx['value']: prop+'='+str(val)}
    
    out_str = ''
    for v in vals.values():
        out_str += str(v) + ' '
        
    return out_str.strip() 

In [132]:
create_main_hdr(f)
add_pin(f, 1, 0, 0, 'Left', False, 48, 'RESET', 'IN')
f.write('E')
f.close()